In [ ]:
!pip install kaggle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#download kaggle.json from your Kaggle account (api token )

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d salader/dogs-vs-cats # Download dataset

In [ ]:
!unzip -q /content/dogs-vs-cats.zip -d /content/dogs-vs-cats # Unzip dataset


In [ ]:

import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory=r"/content/dogs-vs-cats/train",
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(128, 128)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory=r"/content/dogs-vs-cats/test",
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(128, 128)
)

In [ ]:


#augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

def process(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label


# Apply normalization and augmentation to train_ds only
train_ds = train_ds.map(process).map(lambda x, y: (data_augmentation(x, training=True), y))

validation_ds = validation_ds.map(process)



In [ ]:


# Define the CNN model
cnn = Sequential()





cnn.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(128,128,3)))
cnn.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.25))

cnn.add(Flatten())

cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(1, activation='sigmoid'))  # binary classification


In [ ]:
cnn.summary()

In [ ]:
cnn.compile(optimizer='adam' ,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

history=cnn.fit(train_ds, validation_data=validation_ds, epochs=20,callbacks=[early_stop])



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess image
img_path = "/content/R.jpeg"  # Update path as needed
img = image.load_img(img_path, target_size=(128, 128))  # use 128x128 if that's your model input size
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
prediction = cnn.predict(img_array)

# Since it's binary classification, output will be probability
print("Raw prediction:", prediction[0][0])

if prediction[0][0] > 0.5:
    print("Prediction: Dog")
else:
    print("Prediction: Cat")
